In [1]:
import gym
import sys
import pylab
import random
import os
import operator
from collections import deque

from skimage import io, color, transform

import numpy as np

import tensorflow as tf
from keras.layers import Dense,Convolution2D,Flatten,Activation
from keras.optimizers import Adam
from keras.models import Sequential

GAME_TYPE = 'MsPacman-v0'
env =gym.make("MsPacman-v0")

Using TensorFlow backend.
[2017-12-27 20:56:39,028] Making new env: MsPacman-v0


In [2]:
#####
# Hyperparameters
#####

#environment parameters
NUM_EPISODES = 80000000
#MAX_TIMESTEPS = 
#FRAME_SKIP = 
PHI_LENGTH = 4

#agent parameters
#NAIVE_RANDOM = 
EPSILON = 1
EXPERIENCE_REPLAY_CAPACITY = 2000
MINIBATCH_SIZE = 100
LEARNING_RATE = 0.001
ACTION_SIZE = env.action_space.n
EXPLORE = 3000000
print ACTION_SIZE

9


In [3]:
PREPROCESS_IMAGE_DIM = 84

def preprocess_observation(observation):
    """
    preprocesses a given observation following the steps described in the paper
    """
    grayscale_observation = color.rgb2gray(observation)
    resized_observation = transform.resize(grayscale_observation, (PREPROCESS_IMAGE_DIM, PREPROCESS_IMAGE_DIM)).astype('float32')
    return resized_observation

In [4]:
#ignore this cell please
state =env.reset()
#print state
print "State.shape = %s" % (state.shape,)
print "State_size = %s" % (state.size,)
print 210*160*3
#laying out all 100800 features in 1 row, for fun
state = np.reshape(state, [1, state.size])
#print state
print "State.shape now = %s" % (state.shape,)
print "State_size now = %s" % (state.size,)
#print env.observation_space
print "env.observation_space.shape = %s" %(env.observation_space.shape,)

action = random.randrange(env.action_space.n)
next_state, reward, done, info = env.step(action)
#print next_state
print "next_State.shape = %s" % (next_state.shape,)
next_state = np.reshape(state, [1, next_state.size])
#print next_state
print "next_State.shape now = %s" % (next_state.shape,)

#slight change of notations
#ok let next_state be called x_t1 and current state be x_t
# let reward be r_t and action a_t, 9=env.action_space.n= possible actions
x_t = env.reset()
x_t = preprocess_observation(x_t)
print "x_t.shape = %s"% (x_t.shape,)

s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
print "s_t.shape = %s"% (s_t.shape,)
s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*84*84*4
print "s_t.shape now = %s"% (s_t.shape,)

#above stuff - one time per episode at the start eventualy set s_t = s_t1 and so on 

a_t = random.randrange(env.action_space.n)
x_t1 , r_t, done, info = env.step(a_t)
print x_t1.shape

x_t1=preprocess_observation(x_t1)
print "x_t1.shape = %s"% (x_t1.shape,)
x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1],1) #1x84x84x1
print "x_t1.shape now = %s"% (x_t1.shape,)
s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)
print "s_t1.shape = %s"% (s_t1.shape,)
#now s_t =s_t1 and then loop

State.shape = (210, 160, 3)
State_size = 100800
100800
State.shape now = (1, 100800)
State_size now = 100800
env.observation_space.shape = (210, 160, 3)
next_State.shape = (210, 160, 3)
next_State.shape now = (1, 100800)
x_t.shape = (84, 84)
s_t.shape = (84, 84, 4)
s_t.shape now = (1, 84, 84, 4)
(210, 160, 3)
x_t1.shape = (84, 84)
x_t1.shape now = (1, 84, 84, 1)
s_t1.shape = (1, 84, 84, 4)


/home/pranav/lab1/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
#visualising deque
D = deque(maxlen=5)
D.append((x_t, a_t, r_t, x_t1, done))
D.append((s_t, a_t, r_t, s_t1, done))
print D

deque([(array([[ 0.39959559,  0.39959559,  0.39959559, ...,  0.39959559,
         0.39959559,  0.39959559],
       [ 0.53279412,  0.53279412,  0.11700706, ...,  0.11700706,
         0.53279412,  0.53279412],
       [ 0.53279412,  0.53279412,  0.11700706, ...,  0.11700706,
         0.53279412,  0.53279412],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32), 4, 0.0, array([[[[ 0.39959559],
         [ 0.39959559],
         [ 0.39959559],
         ..., 
         [ 0.39959559],
         [ 0.39959559],
         [ 0.39959559]],

        [[ 0.53279412],
         [ 0.53279412],
         [ 0.11700706],
         ..., 
         [ 0.11700706],
         [ 0.53279412],
         [ 0.53279412]],

        [[ 0.53279412],
        

In [6]:
#lets begin
class Agent:

    def __init__(self, epsilon , experience_replay_capacity , minibatch_size , learning_rate ,action_size, img_dim , explore):
        
        self.action_size = action_size
        self.discount_factor = 0.99
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.epsilon_min = 0.01
        self.batch_size = minibatch_size
        self.train_start = 1000
        self.explore = explore
        self.img_rows = img_dim
        self.img_cols = img_dim
        self.img_channels = 4 #phi_length  #coz we feed in 4 stacked b&w imgs instead of 1 rbg img
        
         # create replay memory using deque
        self.D = deque(maxlen=experience_replay_capacity)
        # create main model and target model
        self.model = self.build_model()
        self.target_model = self.build_model()

        # initialize target model
        self.update_target_model()
        
    def build_model(self) :
        model = Sequential()
        model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(self.img_rows,self.img_cols,self.img_channels)))  #84*84*4
        model.add(Activation('relu'))
        model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        model.add(Activation('relu'))
        model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dense(9))
                                
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        #print("finish building the model")
        return model
    # after some time interval update the target model to be same with model
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
        
    def append_experience_replay_example(self,s_t,a_t,r_t,s_t1,done):
        """
        Add an experience replay example to our agent's replay memory. If
        memory is full, overwrite previous examples, starting with the oldest
        """
        D.append((s_t, a_t, r_t, s_t1, done))
       
        if self.epsilon > self.epsilon_min :
            self.epsilon -= (self.epsilon - self.epsilon_min) /self.explore
        
    def preprocess_observation(self, observation, prediction=False):
        """
        Helper function for preprocessing an observation for consumption by our
        deep learning network
        """
        grayscale_observation = color.rgb2gray(observation)
        resized_observation = transform.resize(grayscale_observation, (1, self.processed_image_dim, self.processed_image_dim)).astype('float32')
        if prediction:
            resized_observation = np.expand_dims(resized_observation, 0)
        return resized_observation
        
    def take_action(self, s_t):
        """
        Given an observation, the model attempts to take an action
        according to its q-function approximation
        """
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(s_t)
            return np.argmax(q_value[0])    
        
    def learn(self):
        """
        Allow the model to collect examples from its experience replay memory
        and learn from them
        """
        if len(self.D) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.D))
        mini_batch = random.sample(self.D, batch_size)
        
        update_input = np.zeros((batch_size, s_t.shape[1], s_t.shape[2], s_t.shape[3])) #batch_size,84,84,4
        print(inputs)
        update_target = np.zeros((update_input.shape[0], self.action_size))
        action, reward, done = [], [], []
        
        #Now we do the experience replay
        for i in range(self.batch_size):
            update_input[i] = mini_batch[i][0] # array of s_t that were copied into mini_batch 
            action.append(mini_batch[i][1])    # array of action indices = a_t of actions taken
            reward.append(mini_batch[i][2])
            update_target[i] = mini_batch[i][3] # array of s_t1 that were copied into mini_batch 
            done.append(mini_batch[i][4])
            
        target = self.model.predict(update_input)
        Q_sa = model.predict(update_target) #target_val
        
        for i in range(self.batch_size):
            # Q Learning: get maximum Q value at s' from target model
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.discount_factor * (np.amax(Q_sa[i]))
         # and do the model fit!
        self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)    
                

In [14]:
def run_simulation():
    """
    Entry-point for running Ms. Pac-man simulation
    """



    #print game parameters
    print ("~~~Environment Parameters~~~")
    print ("Num episodes: %s" % NUM_EPISODES)
 #   print ("Max timesteps: %s" % MAX_TIMESTEPS)
    print ("Action space: %s" % env.action_space)
    print()
    print ("~~~Agent Parameters~~~")
 #   print ("Naive Random: %s" % NAIVE_RANDOM)
    print ("Epsilon: %s" % EPSILON)
    print ("Experience Replay Capacity: %s" % EXPERIENCE_REPLAY_CAPACITY)
    print ("Minibatch Size: %s" % MINIBATCH_SIZE)
    print ("Learning Rate: %s" % LEARNING_RATE)

    #initialize agent
    agent = Agent(epsilon=EPSILON,
                experience_replay_capacity=EXPERIENCE_REPLAY_CAPACITY,
                minibatch_size=MINIBATCH_SIZE,
                learning_rate=LEARNING_RATE, action_size =ACTION_SIZE, img_dim =PREPROCESS_IMAGE_DIM ,explore =EXPLORE)
    
    scores, episodes = [], [] #note the s

    #initialize auxiliary data structures
    state_list = [] 
    tot_frames = 0

    for i_episode in range(NUM_EPISODES):
        print ("Episode: %s" % i_episode)
         
        done = False
        score = 0
        x_t = env.reset()
        x_t = preprocess_observation(x_t)   
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
        s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*84*84*4

        
        
        while not done:
            env.render()
           # get action for the current state and go one step in environment
            a_t = agent.take_action(s_t)      
            x_t1 , r_t, done, info = env.step(a_t)
            x_t1=preprocess_observation(x_t1)
            x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1],1) #1x84x84x1
            s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)
            agent.append_experience_replay_example(s_t,a_t,r_t,s_t1,done)
            agent.learn()
            score += r_t
            s_t=s_t1
            
        if done:
            # every episode update the target model to be same with model
            agent.update_target_model() 
            scores.append(score)
            episodes.append(i_episode)
            print( "  score:", score, "  epsilon:", agent.epsilon)
            
        while True:
           
            #ensure state list is populated
            if tot_frames < PHI_LENGTH:
                state_list.append(preprocess_observation(x_t))
                tot_frames += 1
                continue



                #update state list with next observation
                state_list.append(preprocess_observation(x_t))
                state_list.pop(0)


In [ ]:
if __name__ == "__main__":
    env =gym.make("MsPacman-v0")
    run_simulation()

[2017-12-27 21:08:28,542] Making new env: MsPacman-v0


~~~Environment Parameters~~~
Num episodes: 80000000
Action space: Discrete(9)
()
~~~Agent Parameters~~~
Epsilon: 1
Experience Replay Capacity: 2000
Minibatch Size: 100
Learning Rate: 0.001


/home/pranav/lab1/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), padding="same", strides=(4, 4), input_shape=(84, 84, 4...)`
/home/pranav/lab1/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", strides=(2, 2))`
/home/pranav/lab1/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", strides=(1, 1))`


Episode: 0
('  score:', 130.0, '  epsilon:', 0.999814557339431)
